In [1]:
# https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/
# https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069

In [2]:
###########################
# google drive 저장/로딩
###########################
#구글 드라이브 저장
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gcp_drive = None
def doGoodleDriveAuth():
    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    print('gauth:',gauth)
    gauth.credentials = GoogleCredentials.get_application_default()

    gcp_drive = GoogleDrive(gauth)
    print('gcp_drive:', gcp_drive)
    return gcp_drive

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html
def gcp_upload(file_path, w_file_name):
    try:
        if gcp_drive == None:
            drive = doGoodleDriveAuth()

        # 특정 폴더 안으로 파일 삽입
        uploaded = drive.CreateFile({'title': w_file_name}) #, "parents": [{"kind": "drive#fileLink","id": 'jukyellow@gmail.com'}]})
        uploaded.SetContentString(w_file_name)
        uploaded.SetContentFile(file_path + w_file_name)
        uploaded.Upload()
        print('Uploaded file with ID {}'.format(uploaded.get('id')))
        return uploaded.get('id')
    except Exception as e: print('gcp_upload err:', e)

#구글 드라이브에서 다운로드
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
import io
from io import BytesIO 
from googleapiclient.http import MediaIoBaseDownload

TEMP_PATH = '/tmp/'
def gcp_download(file_name, key):
    #3. 모델 다운로드
    #https://drive.google.com/open?id=1TlvbayGRCjAI6bOZrUYMmv6g6b95rnRM
    request = drive_service.files().get_media(fileId=key)

    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()
      if status:
          print("Download %%%d%%." % int(status.progress() * 100))
      print("Download Complete!")
    downloaded.seek(0)

    with open(TEMP_PATH + file_name, 'wb') as f:
        f.write(downloaded.read())

In [3]:
# gcp에서 폴더명 사진 다운로드, 압축풀기
down_file_name = 'data_cat_dog.zip'
gcp_download(down_file_name, '1u9bOu4m6XLDHYqZM26zXjI_2BsbIaePi') #https://drive.google.com/file/d/1u9bOu4m6XLDHYqZM26zXjI_2BsbIaePi/view?usp=sharing

Download %100%.
Download Complete!


In [4]:
!mv /tmp/data_cat_dog.zip /content
# !mv /tmp/KT_전체_디렉토리구분_Te1.zip /content/data/train
# !mv /tmp/전체_디렉토리구분_Test.zip /content/data/test

In [5]:
# 한글깨짐 방지: -O cp949 
!unzip -O cp949 /content/data_cat_dog.zip
!rm -rf /content/data_cat_dog.zip

Archive:  /content/data_cat_dog.zip
   creating: data/test/
   creating: data/test/cat/
  inflating: data/test/cat/cat.12100.jpg  
  inflating: data/test/cat/cat.12101.jpg  
  inflating: data/test/cat/cat.12102.jpg  
  inflating: data/test/cat/cat.12103.jpg  
  inflating: data/test/cat/cat.12104.jpg  
  inflating: data/test/cat/cat.12105.jpg  
  inflating: data/test/cat/cat.12106.jpg  
  inflating: data/test/cat/cat.12107.jpg  
  inflating: data/test/cat/cat.12108.jpg  
  inflating: data/test/cat/cat.12109.jpg  
  inflating: data/test/cat/cat.12110.jpg  
  inflating: data/test/cat/cat.12111.jpg  
  inflating: data/test/cat/cat.12112.jpg  
  inflating: data/test/cat/cat.12113.jpg  
  inflating: data/test/cat/cat.12114.jpg  
  inflating: data/test/cat/cat.12115.jpg  
  inflating: data/test/cat/cat.12116.jpg  
  inflating: data/test/cat/cat.12117.jpg  
  inflating: data/test/cat/cat.12118.jpg  
  inflating: data/test/cat/cat.12119.jpg  
  inflating: data/test/cat/cat.12120.jpg  
  inflati

In [6]:
# gcp에서 폴더명 사진 다운로드, 압축풀기
down_file_name = 'bottleneck_fc_model.h5'
gcp_download(down_file_name, '1KQv3UYEtpckB8tTD9mNuQ5LK29B9baDa') #https://drive.google.com/file/d/1KQv3UYEtpckB8tTD9mNuQ5LK29B9baDa/view?usp=sharing

Download %100%.
Download Complete!


In [7]:
!mv /tmp/bottleneck_fc_model.h5 /content

In [8]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense

# path to the model weights files.
# https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5
weights_path = './vgg16_weights_tf_dim_ordering_tf_kernels.h5' 
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [9]:
# !wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5

In [10]:
ls -alrt

total 8244
drwxr-xr-x 1 root root    4096 Dec 21 17:29 sample_data/
drwxr-xr-x 1 root root    4096 Jan  4 21:30 ../
drwxr-xr-x 1 root root    4096 Jan  4 21:34 .config/
-rw-r--r-- 1 root root     720 Jan  4 21:34 adc.json
drwxr-xr-x 4 root root    4096 Jan  4 21:35 data/
-rw-r--r-- 1 root root 8405536 Jan  4 21:35 bottleneck_fc_model.h5
drwxr-xr-x 1 root root    4096 Jan  4 21:35 ./


In [11]:
# build the VGG16 network
input_shape = (img_width, img_height, 3)
#input_tensor = Input(shape=(150,150,3))

#include_top: whether to include the 3 fully-connected layers at the top of the network.
# weights: one of None (random initialization), 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.
vgg16_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
print('Model loaded.')
print('vgg16_model.output_shape[1:]:', vgg16_model.output_shape[1:])
vgg16_model.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model loaded.
vgg16_model.output_shape[1:]: (4, 4, 512)
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______

In [12]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
# additional_model.add(pre_trained_vgg)
# additional_model.add(layers.Flatten())
top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)
#top_model.load_weights(weights_path)
top_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________


In [13]:
# add the model on top of the convolutional base
#vgg16_model.add(top_model)

#model = Model(input= vgg16_model.input, output= top_model(vgg16_model.output))
model = Model(inputs=vgg16_model.input, outputs=top_model(vgg16_model.output))
#model = Model(vgg16_model.input, top_model.output)
#model = Model(vgg16_model.input, top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False # VGG 하단의 25개 레이어는 동결키십니다

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [14]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [15]:
# Fine tuning 학습을 수행합니다.
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
125/125 [==============================] - 26s 147ms/step - loss: 0.6529 - accuracy: 0.9330 - val_loss: 1.6695 - val_accuracy: 0.8788
Epoch 2/50
125/125 [==============================] - 18s 144ms/step - loss: 0.5896 - accuracy: 0.9316 - val_loss: 1.6695 - val_accuracy: 0.8788
Epoch 3/50
125/125 [==============================] - 18s 146ms/step - loss: 0.8123 - accuracy: 0.9202 - val_loss: 1.6695 - val_accuracy: 0.8788
Epoch 4/50
125/125 [==============================] - 18s 147ms/step - loss: 0.5461 - accuracy: 0.9317 - val_loss: 1.6695 - val_accuracy: 0.8788
Epoch 5/50
125/125 [==============================] - 18s 146ms/step - loss: 0.4234 - accuracy: 0.9498 - val_loss: 1.6695 - val_accuracy: 0.8788
Epoch 6/50
125/125 [==============================] - 18s 146ms/step - loss: 0.5739 - accuracy: 0.9350 - val_loss: 1.6695 - val_accuracy: 0.8788
Epoch 7/50
125/125 [==============================] - 19s 148ms/step - loss: 0.5607 - accuracy: 0.9308 - val_loss: 1.6695 - val_ac